In [ ]:
import silence_tensorflow.auto

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
# Getting numpy data

path_save = os.path.join(os.path.abspath('.'), 'npy')
train_x = np.load(os.path.join(path_save, 'train_img.npy'))
test_x = np.load(os.path.join(path_save, 'test_img.npy'))
train_y = np.load(os.path.join(path_save, 'train_lab.npy'))
test_y = np.load(os.path.join(path_save, 'test_lab.npy'))

# split for validation set
train_x, valid_x, train_y, valid_y= train_test_split(train_x, train_y, test_size=0.2, random_state=4, stratify= train_y)

print('Data shape')
print(f'Train img: {train_x.shape} | Train lab: {train_y.shape}')
print(f'valid img: {valid_x.shape} | valid lab: {valid_y.shape}')
print(f'Test img: {test_x.shape} | Test lab: {test_y.shape}')


In [ ]:
# Using ImageDataGenerator
## Checking output data
train_gen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip = True,
    rotation_range=0.25,
    zoom_range=[1.0, 1.5]
)
train_generator = train_gen.flow(train_x, train_y, shuffle=True, batch_size=4)
augs = train_generator.__getitem__(7)

plt.figure(figsize=(30, 30))
for i, img in enumerate(augs[0]):
    plt.subplot(4, 8, i+1)
    plt.axis('off')
    plt.imshow(img.squeeze(), cmap='gray')

In [ ]:
def Simple_CNNmodel(input_img, base = 32, scale = 2, n_layers = 6):

    x = input_img

    for n in range(n_layers):
        x = Conv2D(((scale)**n)*base, 3, activation=None, padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Activation(activation='relu')(x)
        x = Conv2D(((scale)**n)*base, 3, activation=None, padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Activation(activation='relu')(x)
        if n != n_layers:
            x = MaxPooling2D(pool_size=(2, 2))(x)
        else:
            pass
    
    out = GlobalAveragePooling2D(name="avg_pool")(x)
    out = Dense(128, activation="relu")(out)
    out = BatchNormalization()(out)
    
    out = Dense(64, activation="relu")(out)
    out = BatchNormalization()(out)
    out = Dropout(0.3)(out)
    
    out = Dense(n_class, activation="softmax")(out)
    model = Model(inputs=input_img, outputs=out)
    
    return model
# Training classification model using Simple CNN model with data augmentation

path_save = os.path.join(os.path.abspath('.'), 'Result', 'model_simple_augmentation')
os.makedirs(path_save, exist_ok=True)

## parameter setting
n_class = 2         
imageSize = 512     
lr = 0.0001
epochs = 50
batch = 10
loss_function = "categorical_crossentropy"

## model build
input_img = Input(shape=(imageSize,imageSize,1))

model = Simple_CNNmodel(input_img, n_class)
model.summary()         # Check model structure

## Data generator for augmentation
train_gen = ImageDataGenerator(
    width_shift_range=0.2,       # 이미지 너비의 20% 범위 내에서 임의로 좌우 이동
    height_shift_range=0.2,     # 이미지 높이의 20% 범위 내에서 임의로 좌우 이동
    horizontal_flip=True,       # 수평 뒤집기
    vertical_flip = True,       # 수직 뒤집기
    rotation_range=0.25,        # 중심축을 기준으로 최대 ±0.25도 회전
    zoom_range=[0.5, 1.5]       # 최대 0.5배 축소부터 1.5배 확대
)
valid_gen = ImageDataGenerator()        


## Callback
checkpointer = ModelCheckpoint(filepath=os.path.join(path_save,f'best_model.h5'), 
                                verbose=1, 
                                save_weights_only=True, 
                                save_best_only=True, 
                                monitor='val_loss', 
                                save_freq='epoch')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0, min_delta=0.001, verbose=1)
callbacks_list = [reduce_lr, checkpointer]

## model compile
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(
        optimizer=optimizer, loss=loss_function, metrics=["accuracy"]
    )

## training model
model.fit(train_gen.flow(train_x, train_y, shuffle=True, batch_size=batch), 
            steps_per_epoch=len(train_x) // batch, 
            epochs=epochs,
            validation_data=valid_gen.flow(valid_x, valid_y, batch_size=batch, shuffle=False),
            callbacks=callbacks_list
                        )

## saving last model weight
model.save_weights(os.path.join(path_save,f'last_model.h5'))


In [ ]:
# Evaluation using train data
_loss, _acc = model.evaluate(train_x, train_y, batch_size=10, verbose=1)
print('Last model accuracy')
print(f'loss: {_loss:0.3f} | accuracy: {_acc:0.3f}')


In [ ]:
# Evaluation using test data
_loss, _acc = model.evaluate(test_x, test_y, batch_size=10, verbose=1)
print('Last model accuracy')
print(f'loss: {_loss:0.3f} | accuracy: {_acc:0.3f}')


In [ ]:
## Using best model
model.load_weights(os.path.join(path_save,f'best_model.h5'))
_loss, _acc = model.evaluate(test_x, test_y, batch_size=10, verbose=1)
print('Best model accuracy')
print(f'loss: {_loss:0.3f} | accuracy: {_acc:0.3f}')

In [ ]:
# Checking fail data
test_result = model.predict(test_x, batch_size=10)

for a in range(len(test_result)):
    if test_result[a].argmax()!=test_y[a].argmax():
        plt.title(f'GT: {test_y[a].argmax()} | Result: {test_result[a].argmax()}')
        plt.imshow(test_x[a], cmap='gray')
        plt.show()

In [ ]:
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
print(classification_report(test_y.argmax(axis=1), test_result.argmax(axis=1)))

# Comufsion matrix 확인

test_th = np.where(test_result > 0.5, 1, 0)

cm = confusion_matrix(test_y.argmax(axis=1), test_result.argmax(axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['normal', 'abnormal'])
disp.plot()